In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

def load_data(filepath):
    """Load the raw financial dataset"""
    print(" Loading dataset...")
    df = pd.read_csv(filepath)
    print(f" Loaded {len(df)} transactions")
    return df

def parse_dates(df):
    """Parse dates from string to datetime"""
    print("\n Parsing dates...")
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')
    print(f" Date range: {df['Date'].min()} to {df['Date'].max()}")
    return df

def create_time_features(df):
    """Create time-based features"""
    print("\n Creating time-based features...")

    # Basic time components
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Month_Name'] = df['Date'].dt.strftime('%B')
    df['Quarter'] = df['Date'].dt.quarter
    df['Day'] = df['Date'].dt.day
    df['Day_of_Week'] = df['Date'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['Day_Name'] = df['Date'].dt.strftime('%A')
    df['Week_of_Year'] = df['Date'].dt.isocalendar().week

    # Week of month (1-5)
    df['Week_of_Month'] = ((df['Day'] - 1) // 7) + 1

    # Binary flags
    df['Is_Weekend'] = df['Day_of_Week'].isin([5, 6]).astype(int)
    df['Is_Month_Start'] = df['Date'].dt.is_month_start.astype(int)
    df['Is_Month_End'] = df['Date'].dt.is_month_end.astype(int)

    print(" Created 13 time-based features")
    return df

def create_amount_features(df):
    """Create amount-based features"""
    print("\n Creating amount-based features...")

    # Sort by date first
    df = df.sort_values('Date').reset_index(drop=True)

    # Lag feature
    df['Previous_Transaction_Amount'] = df['Amount'].shift(1)

    # Rolling averages
    df['Rolling_7day_Avg'] = df['Amount'].rolling(window=7, min_periods=1).mean()
    df['Rolling_30day_Avg'] = df['Amount'].rolling(window=30, min_periods=1).mean()

    # Category average
    category_avg = df.groupby('Category')['Amount'].mean().to_dict()
    df['Category_Avg_Amount'] = df['Category'].map(category_avg)

    # Days since last transaction
    df['Days_Since_Last_Transaction'] = df['Date'].diff().dt.days
    df.loc[0, 'Days_Since_Last_Transaction'] = 0

    # Cumulative spending by category
    df['Cumulative_Spending_By_Category'] = df.groupby('Category')['Amount'].cumsum()

    # Transaction count by category
    df['Transaction_Count_By_Category'] = df.groupby('Category').cumcount() + 1

    print(" Created 7 amount-based features")
    return df

def create_categorical_features(df):
    """Create categorical features"""
    print("\n Creating categorical features...")

    # Income flag
    df['Is_Income'] = (df['Type'] == 'Income').astype(int)

    # Monthly aggregations
    df['Year_Month'] = df['Date'].dt.to_period('M')
    monthly_totals = df.groupby(['Year_Month', 'Category'])['Amount'].sum().reset_index()
    monthly_totals.columns = ['Year_Month', 'Category', 'Monthly_Total_By_Category']
    df = df.merge(monthly_totals, on=['Year_Month', 'Category'], how='left')

    # Clean descriptions
    df['Transaction_Description_Clean'] = df['Transaction Description'].str.strip()

    print(" Created 3 categorical features")
    return df

def reorder_columns(df):
    """Reorder columns for better readability"""
    print("\n Reordering columns...")

    column_order = [
        'Date', 'Year', 'Month', 'Month_Name', 'Quarter', 'Day', 'Day_of_Week', 'Day_Name',
        'Week_of_Year', 'Week_of_Month', 'Is_Weekend', 'Is_Month_Start', 'Is_Month_End',
        'Transaction Description', 'Transaction_Description_Clean', 'Category', 'Amount', 'Type', 'Is_Income',
        'Category_Avg_Amount', 'Monthly_Total_By_Category',
        'Previous_Transaction_Amount', 'Rolling_7day_Avg', 'Rolling_30day_Avg',
        'Days_Since_Last_Transaction', 'Cumulative_Spending_By_Category', 'Transaction_Count_By_Category'
    ]

    df = df[column_order]
    print(" Columns reordered")
    return df

def generate_summary(df):
    """Generate summary statistics"""
    print("\n" + "="*70)
    print(" SUMMARY STATISTICS")
    print("="*70)

    print(f"\n Dataset Info:")
    print(f"   Total Transactions: {len(df):,}")
    print(f"   Total Features: {len(df.columns)}")
    print(f"   Date Range: {df['Date'].min().strftime('%Y-%m-%d')} to {df['Date'].max().strftime('%Y-%m-%d')}")

    print(f"\n Financial Summary:")
    total_income = df[df['Type'] == 'Income']['Amount'].sum()
    total_expense = df[df['Type'] == 'Expense']['Amount'].sum()
    net = total_income - total_expense

    print(f"   Total Income:  ${total_income:>15,.2f}")
    print(f"   Total Expense: ${total_expense:>15,.2f}")
    print(f"   Net:           ${net:>15,.2f}")

    print(f"\n Transaction Types:")
    print(f"   Income:  {len(df[df['Type'] == 'Income']):>5} transactions")
    print(f"   Expense: {len(df[df['Type'] == 'Expense']):>5} transactions")

    print(f"\n Categories:")
    print(f"   Unique Categories: {df['Category'].nunique()}")
    for cat in sorted(df['Category'].unique()):
        count = len(df[df['Category'] == cat])
        print(f"   - {cat:<20} {count:>4} transactions")

    print(f"\n Amount Statistics:")
    print(f"   Mean:   ${df['Amount'].mean():,.2f}")
    print(f"   Median: ${df['Amount'].median():,.2f}")
    print(f"   Min:    ${df['Amount'].min():,.2f}")
    print(f"   Max:    ${df['Amount'].max():,.2f}")
    print(f"   Std:    ${df['Amount'].std():,.2f}")

def save_data(df, output_path):
    """Save cleaned dataset"""
    print(f"\n Saving cleaned dataset...")
    df.to_csv(output_path, index=False)
    print(f" Saved to: {output_path}")

def main():
    """Main execution function"""
    print("="*70)
    print("🎯 PERSONAL FINANCE INTELLIGENCE PLATFORM")
    print("   Phase 1: Data Cleaning & Feature Engineering")
    print("="*70)

    # Configuration
    INPUT_FILE = '/content/drive/My Drive/project_p1/Personal_Finance_Dataset.csv'
    OUTPUT_FILE = '/content/drive/My Drive/project_p1/Personal_Finance_CLEANED.csv'

    try:
        # Step 1: Load data
        df = load_data(INPUT_FILE)

        # Step 2: Parse dates
        df = parse_dates(df)

        # Step 3: Create time features
        df = create_time_features(df)

        # Step 4: Create amount features
        df = create_amount_features(df)

        # Step 5: Create categorical features
        df = create_categorical_features(df)

        # Step 6: Reorder columns
        df = reorder_columns(df)

        # Step 7: Generate summary
        generate_summary(df)

        # Step 8: Save data
        save_data(df, OUTPUT_FILE)

        print("\n" + "="*70)
        print(" PHASE 1 COMPLETED SUCCESSFULLY!")
        print("="*70)
        print("\n Next Steps:")
        print("   1. Review the cleaned dataset")
        print("   2. Upload to Google Colab for ML analysis")
        print("   3. Proceed to Phase 2: Machine Learning Models")
        print("\n" + "="*70)

    except FileNotFoundError:
        print(f"\n Error: File '{INPUT_FILE}' not found!")
        print("   Please make sure the file exists in the current directory.")
    except Exception as e:
        print(f"\n Error occurred: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

🎯 PERSONAL FINANCE INTELLIGENCE PLATFORM
   Phase 1: Data Cleaning & Feature Engineering
 Loading dataset...
 Loaded 1500 transactions

 Parsing dates...
 Date range: 2020-01-02 00:00:00 to 2024-12-29 00:00:00

 Creating time-based features...
 Created 13 time-based features

 Creating amount-based features...
 Created 7 amount-based features

 Creating categorical features...
 Created 3 categorical features

 Reordering columns...
 Columns reordered

 SUMMARY STATISTICS

 Dataset Info:
   Total Transactions: 1,500
   Total Features: 27
   Date Range: 2020-01-02 to 2024-12-29

 Financial Summary:
   Total Income:  $     734,087.00
   Total Expense: $   1,227,194.37
   Net:           $    -493,107.37

 Transaction Types:
   Income:    278 transactions
   Expense:  1222 transactions

 Categories:
   Unique Categories: 10
   - Entertainment         143 transactions
   - Food & Drink          149 transactions
   - Health & Fitness      152 transactions
   - Investment            142 transa

In [3]:
"""
PERSONAL FINANCE INTELLIGENCE PLATFORM
SQL Database Creation Script

This script creates a SQLite database from the cleaned CSV data
and sets up proper tables with indexes for fast queries.

"""

import pandas as pd
import sqlite3
from datetime import datetime
import os

def create_database(csv_file, db_file='personal_finance.db'):
    """Create SQLite database from CSV file"""

    print("="*70)
    print("  CREATING SQL DATABASE")
    print("="*70)

    # Load cleaned data
    print("\n Loading cleaned data...")
    df = pd.read_csv(csv_file)
    df['Date'] = pd.to_datetime(df['Date'])
    print(f" Loaded {len(df)} transactions")

    # Remove existing database if it exists
    if os.path.exists(db_file):
        os.remove(db_file)
        print(f"  Removed existing database: {db_file}")

    # Create connection
    print(f"\n Creating database: {db_file}")
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # =========================================================================
    # TABLE 1: TRANSACTIONS (Main table)
    # =========================================================================
    print("\n Creating TRANSACTIONS table...")

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS transactions (
        transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
        date DATE NOT NULL,
        year INTEGER,
        month INTEGER,
        month_name TEXT,
        quarter INTEGER,
        day INTEGER,
        day_of_week INTEGER,
        day_name TEXT,
        week_of_year INTEGER,
        week_of_month INTEGER,
        is_weekend INTEGER,
        is_month_start INTEGER,
        is_month_end INTEGER,
        description TEXT,
        description_clean TEXT,
        category TEXT NOT NULL,
        amount REAL NOT NULL,
        type TEXT NOT NULL,
        is_income INTEGER,
        category_avg_amount REAL,
        monthly_total_by_category REAL,
        previous_transaction_amount REAL,
        rolling_7day_avg REAL,
        rolling_30day_avg REAL,
        days_since_last_transaction REAL,
        cumulative_spending_by_category REAL,
        transaction_count_by_category INTEGER
    )
    ''')

    # Insert data
    print(" Inserting transaction data...")
    df_to_insert = df.copy()
    df_to_insert['Date'] = df_to_insert['Date'].dt.strftime('%Y-%m-%d')

    df_to_insert.to_sql('transactions', conn, if_exists='replace', index=False)

    # Recreate primary key
    cursor.execute('''
    CREATE TABLE transactions_new (
        transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
        date DATE NOT NULL,
        year INTEGER,
        month INTEGER,
        month_name TEXT,
        quarter INTEGER,
        day INTEGER,
        day_of_week INTEGER,
        day_name TEXT,
        week_of_year INTEGER,
        week_of_month INTEGER,
        is_weekend INTEGER,
        is_month_start INTEGER,
        is_month_end INTEGER,
        description TEXT,
        description_clean TEXT,
        category TEXT NOT NULL,
        amount REAL NOT NULL,
        type TEXT NOT NULL,
        is_income INTEGER,
        category_avg_amount REAL,
        monthly_total_by_category REAL,
        previous_transaction_amount REAL,
        rolling_7day_avg REAL,
        rolling_30day_avg REAL,
        days_since_last_transaction REAL,
        cumulative_spending_by_category REAL,
        transaction_count_by_category INTEGER
    )
    ''')

    cursor.execute('''
    INSERT INTO transactions_new
    SELECT NULL, * FROM transactions
    ''')

    cursor.execute('DROP TABLE transactions')
    cursor.execute('ALTER TABLE transactions_new RENAME TO transactions')

    print(f" Inserted {len(df)} transactions")

    # =========================================================================
    # TABLE 2: CATEGORIES (Lookup table)
    # =========================================================================
    print("\n Creating CATEGORIES table...")

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS categories (
        category_id INTEGER PRIMARY KEY AUTOINCREMENT,
        category_name TEXT UNIQUE NOT NULL,
        category_type TEXT
    )
    ''')

    categories = df['Category'].unique()
    for cat in categories:
        cursor.execute('''
        INSERT OR IGNORE INTO categories (category_name, category_type)
        VALUES (?, ?)
        ''', (cat, 'Expense'))

    print(f" Inserted {len(categories)} categories")

    # =========================================================================
    # TABLE 3: MONTHLY_SUMMARY (Aggregated table)
    # =========================================================================
    print("\n Creating MONTHLY_SUMMARY table...")

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS monthly_summary (
        summary_id INTEGER PRIMARY KEY AUTOINCREMENT,
        year INTEGER,
        month INTEGER,
        month_name TEXT,
        total_income REAL,
        total_expense REAL,
        net_amount REAL,
        transaction_count INTEGER,
        avg_transaction_amount REAL,
        savings_rate REAL
    )
    ''')

    # Calculate monthly summary
    monthly_data = df.groupby(['Year', 'Month', 'Month_Name']).agg({
        'Amount': ['sum', 'count', 'mean']
    }).reset_index()

    monthly_data.columns = ['year', 'month', 'month_name', 'total', 'count', 'avg']

    # Calculate income and expense separately
    for _, row in monthly_data.iterrows():
        year, month = row['year'], row['month']

        income = df[(df['Year'] == year) & (df['Month'] == month) & (df['Type'] == 'Income')]['Amount'].sum()
        expense = df[(df['Year'] == year) & (df['Month'] == month) & (df['Type'] == 'Expense')]['Amount'].sum()
        net = income - expense
        savings_rate = (net / income * 100) if income > 0 else 0

        cursor.execute('''
        INSERT INTO monthly_summary
        (year, month, month_name, total_income, total_expense, net_amount,
         transaction_count, avg_transaction_amount, savings_rate)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (year, month, row['month_name'], income, expense, net,
              row['count'], row['avg'], savings_rate))

    print(f" Created monthly summary records")

    # =========================================================================
    # TABLE 4: CATEGORY_SUMMARY (Category aggregations)
    # =========================================================================
    print("\n Creating CATEGORY_SUMMARY table...")

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS category_summary (
        category TEXT PRIMARY KEY,
        total_amount REAL,
        transaction_count INTEGER,
        avg_amount REAL,
        min_amount REAL,
        max_amount REAL,
        percentage_of_total REAL
    )
    ''')

    # Calculate category summary
    total_expense = df[df['Type'] == 'Expense']['Amount'].sum()

    for cat in categories:
        cat_data = df[(df['Category'] == cat) & (df['Type'] == 'Expense')]

        if len(cat_data) > 0:
            total = cat_data['Amount'].sum()
            count = len(cat_data)
            avg = cat_data['Amount'].mean()
            min_amt = cat_data['Amount'].min()
            max_amt = cat_data['Amount'].max()
            pct = (total / total_expense * 100) if total_expense > 0 else 0

            cursor.execute('''
            INSERT INTO category_summary
            (category, total_amount, transaction_count, avg_amount,
             min_amount, max_amount, percentage_of_total)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (cat, total, count, avg, min_amt, max_amt, pct))

    print(f" Created category summary records")

    # =========================================================================
    # CREATE INDEXES for faster queries
    # =========================================================================
    print("\n⚡ Creating indexes...")

    cursor.execute('CREATE INDEX IF NOT EXISTS idx_date ON transactions(date)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_category ON transactions(category)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_type ON transactions(type)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_year ON transactions(year)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_month ON transactions(month)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_year_month ON transactions(year, month)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_amount ON transactions(amount)')

    print(" Created 7 indexes")

    # =========================================================================
    # CREATE VIEWS for common queries
    # =========================================================================
    print("\n👁️  Creating views...")

    # View 1: Income transactions
    cursor.execute('''
    CREATE VIEW IF NOT EXISTS view_income AS
    SELECT * FROM transactions WHERE type = 'Income'
    ''')

    # View 2: Expense transactions
    cursor.execute('''
    CREATE VIEW IF NOT EXISTS view_expenses AS
    SELECT * FROM transactions WHERE type = 'Expense'
    ''')

    # View 3: Current year transactions
    cursor.execute('''
    CREATE VIEW IF NOT EXISTS view_current_year AS
    SELECT * FROM transactions WHERE year = (SELECT MAX(year) FROM transactions)
    ''')

    # View 4: Weekend transactions
    cursor.execute('''
    CREATE VIEW IF NOT EXISTS view_weekend_spending AS
    SELECT * FROM transactions WHERE is_weekend = 1 AND type = 'Expense'
    ''')

    print(" Created 4 views")

    # Commit and close
    conn.commit()

    # =========================================================================
    # DISPLAY SUMMARY
    # =========================================================================
    print("\n" + "="*70)
    print(" DATABASE SUMMARY")
    print("="*70)

    # Table counts
    cursor.execute("SELECT COUNT(*) FROM transactions")
    trans_count = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM categories")
    cat_count = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM monthly_summary")
    monthly_count = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM category_summary")
    cat_sum_count = cursor.fetchone()[0]

    print(f"\n Tables Created:")
    print(f"   1. transactions        - {trans_count:,} records")
    print(f"   2. categories          - {cat_count} records")
    print(f"   3. monthly_summary     - {monthly_count} records")
    print(f"   4. category_summary    - {cat_sum_count} records")

    print(f"\n⚡ Indexes Created: 7")
    print(f"👁️  Views Created: 4")

    # Database size
    db_size = os.path.getsize(db_file) / 1024  # KB
    print(f"\n Database Size: {db_size:.2f} KB")
    print(f"📁 Database File: {db_file}")

    conn.close()

    print("\n" + "="*70)
    print(" DATABASE CREATED SUCCESSFULLY!")
    print("="*70)

    return db_file

def verify_database(db_file='personal_finance.db'):
    """Verify database creation"""
    print("\n🔍 Verifying database...")

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # List all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    print(f"\n Tables: {[t[0] for t in tables]}")

    # Sample query
    cursor.execute("SELECT * FROM transactions LIMIT 5")
    print(f"\n Sample data retrieved successfully")

    conn.close()

if __name__ == "__main__":
    # Configuration
    CSV_FILE = '/content/drive/My Drive/project_p1/Personal_Finance_CLEANED.csv'
    DB_FILE = '/content/drive/My Drive/project_p1/personal_finance.db'

    try:
        # Create database
        db_file = create_database(CSV_FILE, DB_FILE)

        # Verify
        verify_database(db_file)

        print("\n Next Steps:")
        print("   1. Run the SQL queries (use sql_queries.sql)")
        print("   2. Use database_analysis.py for Python queries")
        print("   3. Connect with any SQL client (DB Browser, DBeaver, etc.)")

    except FileNotFoundError:
        print(f"\n Error: '{CSV_FILE}' not found!")
        print("   Make sure Personal_Finance_CLEANED.csv is in the current directory")
    except Exception as e:
        print(f"\n Error: {str(e)}")
        import traceback
        traceback.print_exc()

  CREATING SQL DATABASE

 Loading cleaned data...
 Loaded 1500 transactions
  Removed existing database: /content/drive/My Drive/project_p1/personal_finance.db

 Creating database: /content/drive/My Drive/project_p1/personal_finance.db

 Creating TRANSACTIONS table...
 Inserting transaction data...
 Inserted 1500 transactions

 Creating CATEGORIES table...
 Inserted 10 categories

 Creating MONTHLY_SUMMARY table...
 Created monthly summary records

 Creating CATEGORY_SUMMARY table...
 Created category summary records

⚡ Creating indexes...
 Created 7 indexes

👁️  Creating views...
 Created 4 views

 DATABASE SUMMARY

 Tables Created:
   1. transactions        - 1,500 records
   2. categories          - 10 records
   3. monthly_summary     - 60 records
   4. category_summary    - 8 records

⚡ Indexes Created: 7
👁️  Views Created: 4

 Database Size: 560.00 KB
📁 Database File: /content/drive/My Drive/project_p1/personal_finance.db

 DATABASE CREATED SUCCESSFULLY!

🔍 Verifying database...


In [4]:
"""
PERSONAL FINANCE INTELLIGENCE PLATFORM
Database Analysis Script

Run SQL queries programmatically and export results

"""

import sqlite3
import pandas as pd
from datetime import datetime
import os

class FinanceDatabaseAnalyzer:
    """Class to analyze personal finance database"""

    def __init__(self, db_file='personal_finance.db'):
        """Initialize connection to database"""
        self.db_file = db_file
        self.conn = None

    def connect(self):
        """Connect to database"""
        try:
            self.conn = sqlite3.connect(self.db_file)
            print(f" Connected to {self.db_file}")
            return True
        except Exception as e:
            print(f" Error connecting to database: {e}")
            return False

    def disconnect(self):
        """Close database connection"""
        if self.conn:
            self.conn.close()
            print(" Database connection closed")

    def execute_query(self, query, params=None):
        """Execute a SQL query and return results as DataFrame"""
        try:
            if params:
                df = pd.read_sql_query(query, self.conn, params=params)
            else:
                df = pd.read_sql_query(query, self.conn)
            return df
        except Exception as e:
            print(f" Query error: {e}")
            return None

    # ========================================================================
    # PRE-BUILT QUERY FUNCTIONS
    # ========================================================================

    def get_all_transactions(self, limit=100):
        """Get all transactions"""
        query = f"""
        SELECT * FROM transactions
        ORDER BY date DESC
        LIMIT {limit}
        """
        return self.execute_query(query)

    def get_income_vs_expense(self):
        """Get income vs expense summary"""
        query = """
        SELECT
            type,
            COUNT(*) AS count,
            SUM(amount) AS total,
            AVG(amount) AS average,
            MIN(amount) AS minimum,
            MAX(amount) AS maximum
        FROM transactions
        GROUP BY type
        """
        return self.execute_query(query)

    def get_net_amount(self):
        """Calculate net income (income - expense)"""
        query = """
        SELECT
            SUM(CASE WHEN type = 'Income' THEN amount ELSE 0 END) AS total_income,
            SUM(CASE WHEN type = 'Expense' THEN amount ELSE 0 END) AS total_expense,
            SUM(CASE WHEN type = 'Income' THEN amount ELSE -amount END) AS net_amount
        FROM transactions
        """
        return self.execute_query(query)

    def get_spending_by_category(self):
        """Get spending breakdown by category"""
        query = """
        SELECT
            category,
            COUNT(*) AS transactions,
            SUM(amount) AS total_spent,
            AVG(amount) AS avg_spent,
            ROUND(SUM(amount) * 100.0 / (SELECT SUM(amount) FROM transactions WHERE type = 'Expense'), 2) AS percentage
        FROM transactions
        WHERE type = 'Expense'
        GROUP BY category
        ORDER BY total_spent DESC
        """
        return self.execute_query(query)

    def get_top_expenses(self, n=10):
        """Get top N highest expenses"""
        query = f"""
        SELECT date, description_clean, category, amount
        FROM transactions
        WHERE type = 'Expense'
        ORDER BY amount DESC
        LIMIT {n}
        """
        return self.execute_query(query)

    def get_monthly_summary(self):
        """Get monthly income and expense summary"""
        query = """
        SELECT * FROM monthly_summary
        ORDER BY year, month
        """
        return self.execute_query(query)

    def get_yearly_comparison(self):
        """Get yearly comparison"""
        query = """
        SELECT
            year,
            COUNT(*) AS transactions,
            SUM(CASE WHEN type = 'Income' THEN amount ELSE 0 END) AS income,
            SUM(CASE WHEN type = 'Expense' THEN amount ELSE 0 END) AS expense,
            SUM(CASE WHEN type = 'Income' THEN amount ELSE -amount END) AS net
        FROM transactions
        GROUP BY year
        ORDER BY year
        """
        return self.execute_query(query)

    def get_day_of_week_analysis(self):
        """Analyze spending by day of week"""
        query = """
        SELECT
            day_name,
            day_of_week,
            COUNT(*) AS transactions,
            SUM(amount) AS total_spent,
            AVG(amount) AS avg_spent
        FROM transactions
        WHERE type = 'Expense'
        GROUP BY day_name, day_of_week
        ORDER BY day_of_week
        """
        return self.execute_query(query)

    def get_weekend_vs_weekday(self):
        """Compare weekend vs weekday spending"""
        query = """
        SELECT
            CASE WHEN is_weekend = 1 THEN 'Weekend' ELSE 'Weekday' END AS day_type,
            COUNT(*) AS transactions,
            SUM(amount) AS total_spent,
            AVG(amount) AS avg_spent
        FROM transactions
        WHERE type = 'Expense'
        GROUP BY is_weekend
        """
        return self.execute_query(query)

    def get_category_summary(self):
        """Get category summary table"""
        query = """
        SELECT * FROM category_summary
        ORDER BY total_amount DESC
        """
        return self.execute_query(query)

    def get_unusual_transactions(self, multiplier=2):
        """Find transactions above average by specified multiplier"""
        query = f"""
        SELECT
            date,
            description_clean,
            category,
            amount,
            ROUND((amount / (SELECT AVG(amount) FROM transactions WHERE type = 'Expense')), 2) AS times_average
        FROM transactions
        WHERE type = 'Expense'
          AND amount > (SELECT AVG(amount) * {multiplier} FROM transactions WHERE type = 'Expense')
        ORDER BY amount DESC
        """
        return self.execute_query(query)

    def get_transactions_by_date_range(self, start_date, end_date):
        """Get transactions within date range"""
        query = """
        SELECT * FROM transactions
        WHERE date BETWEEN ? AND ?
        ORDER BY date DESC
        """
        return self.execute_query(query, params=(start_date, end_date))

    def get_transactions_by_category(self, category):
        """Get all transactions for a specific category"""
        query = """
        SELECT * FROM transactions
        WHERE category = ?
        ORDER BY date DESC
        """
        return self.execute_query(query, params=(category,))

    def get_savings_rate(self):
        """Calculate savings rate"""
        query = """
        SELECT
            SUM(CASE WHEN type = 'Income' THEN amount ELSE 0 END) AS income,
            SUM(CASE WHEN type = 'Expense' THEN amount ELSE 0 END) AS expense,
            SUM(CASE WHEN type = 'Income' THEN amount ELSE -amount END) AS savings,
            ROUND(
                (SUM(CASE WHEN type = 'Income' THEN amount ELSE -amount END) * 100.0) /
                SUM(CASE WHEN type = 'Income' THEN amount ELSE 0 END), 2
            ) AS savings_rate_pct
        FROM transactions
        """
        return self.execute_query(query)

    def get_rolling_averages(self, limit=30):
        """Get transactions with rolling averages"""
        query = f"""
        SELECT
            date,
            category,
            amount,
            ROUND(rolling_7day_avg, 2) AS rolling_7day_avg,
            ROUND(rolling_30day_avg, 2) AS rolling_30day_avg
        FROM transactions
        WHERE type = 'Expense'
        ORDER BY date DESC
        LIMIT {limit}
        """
        return self.execute_query(query)

    def get_dashboard_summary(self):
        """Get quick dashboard summary"""
        query = """
        SELECT
            'Total Transactions' AS metric, CAST(COUNT(*) AS TEXT) AS value FROM transactions
        UNION ALL
        SELECT 'Total Income', PRINTF('$%,.2f', SUM(amount)) FROM transactions WHERE type = 'Income'
        UNION ALL
        SELECT 'Total Expense', PRINTF('$%,.2f', SUM(amount)) FROM transactions WHERE type = 'Expense'
        UNION ALL
        SELECT 'Net Amount', PRINTF('$%,.2f', SUM(CASE WHEN type = 'Income' THEN amount ELSE -amount END)) FROM transactions
        UNION ALL
        SELECT 'Categories', CAST(COUNT(DISTINCT category) AS TEXT) FROM transactions
        UNION ALL
        SELECT 'Date Range', MIN(date) || ' to ' || MAX(date) FROM transactions
        """
        return self.execute_query(query)

    # ========================================================================
    # EXPORT FUNCTIONS
    # ========================================================================

    def export_to_csv(self, df, filename):
        """Export DataFrame to CSV"""
        try:
            df.to_csv(filename, index=False)
            print(f" Exported to {filename}")
        except Exception as e:
            print(f" Export error: {e}")

    def export_all_queries(self, output_dir='query_results'):
        """Run all queries and export to CSV files"""
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        print(f"\n Running all queries and exporting to {output_dir}/...")

        queries = {
            'all_transactions': self.get_all_transactions,
            'income_vs_expense': self.get_income_vs_expense,
            'net_amount': self.get_net_amount,
            'spending_by_category': self.get_spending_by_category,
            'top_expenses': self.get_top_expenses,
            'monthly_summary': self.get_monthly_summary,
            'yearly_comparison': self.get_yearly_comparison,
            'day_of_week_analysis': self.get_day_of_week_analysis,
            'weekend_vs_weekday': self.get_weekend_vs_weekday,
            'category_summary': self.get_category_summary,
            'unusual_transactions': self.get_unusual_transactions,
            'savings_rate': self.get_savings_rate,
            'dashboard_summary': self.get_dashboard_summary
        }

        for name, func in queries.items():
            df = func()
            if df is not None:
                filename = os.path.join(output_dir, f'{name}.csv')
                self.export_to_csv(df, filename)

        print(f"\n All queries exported to {output_dir}/")

    # ========================================================================
    # DISPLAY FUNCTIONS
    # ========================================================================

    def display_summary(self):
        """Display comprehensive summary"""
        print("\n" + "="*70)
        print(" PERSONAL FINANCE DATABASE SUMMARY")
        print("="*70)

        # Dashboard
        print("\n Quick Dashboard:")
        dashboard = self.get_dashboard_summary()
        for _, row in dashboard.iterrows():
            print(f"   {row['metric']:<25} {row['value']}")

        # Income vs Expense
        print("\n Income vs Expense:")
        inc_exp = self.get_income_vs_expense()
        print(inc_exp.to_string(index=False))

        # Savings Rate
        print("\n Savings Rate:")
        savings = self.get_savings_rate()
        print(savings.to_string(index=False))

        # Top 5 Categories
        print("\n Top 5 Spending Categories:")
        categories = self.get_spending_by_category().head(5)
        print(categories.to_string(index=False))

        print("\n" + "="*70)

def main():
    """Main execution function"""
    print("="*70)
    print("  PERSONAL FINANCE DATABASE ANALYZER")
    print("="*70)

    # Initialize analyzer

    analyzer = FinanceDatabaseAnalyzer('/content/drive/My Drive/project_p1/personal_finance.db')
    # Connect to database
    if not analyzer.connect():
        return

    try:
        # Display summary
        analyzer.display_summary()

        # Export all queries
        print("\n" + "="*70)
        user_input = input("\nExport all query results to CSV? (y/n): ")

        if user_input.lower() == 'y':
            analyzer.export_all_queries()

        print("\n" + "="*70)
        print(" ANALYSIS COMPLETE!")
        print("="*70)

        print("\n🎯 Available Methods:")
        print("   - get_all_transactions()")
        print("   - get_income_vs_expense()")
        print("   - get_spending_by_category()")
        print("   - get_monthly_summary()")
        print("   - get_top_expenses(n=10)")
        print("   - get_unusual_transactions(multiplier=2)")
        print("   - get_transactions_by_date_range(start, end)")
        print("   - get_savings_rate()")
        print("   - export_all_queries()")

    finally:
        # Disconnect
        analyzer.disconnect()

if __name__ == "__main__":
    main()

  PERSONAL FINANCE DATABASE ANALYZER
 Connected to /content/drive/My Drive/project_p1/personal_finance.db

 PERSONAL FINANCE DATABASE SUMMARY

 Quick Dashboard:
   Total Transactions        1500
   Total Income              $734087.00
   Total Expense             $1227194.37
   Net Amount                $-493107.37
   Categories                10
   Date Range                2020-01-02 to 2024-12-29

 Income vs Expense:
   type  count      total     average  minimum  maximum
Expense   1222 1227194.37 1004.250712    14.37  1999.82
 Income    278  734087.00 2640.600719   500.00  4996.00

 Savings Rate:
  income    expense    savings  savings_rate_pct
734087.0 1227194.37 -493107.37            -67.17

 Top 5 Spending Categories:
     category  transactions  total_spent   avg_spent  percentage
       Travel           160    169497.79 1059.361188       13.81
         Rent           165    162075.39  982.275091       13.21
 Food & Drink           149    159493.39 1070.425436       13.00
     